# Exercises

In [2]:
# imports
import pyspark
import pandas as pd
import numpy as np
from pydataset import data
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_extract, regexp_replace
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/25 12:03:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/25 12:03:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [264]:
df = pd.DataFrame({'language': ['python', 'java', 'typescript', 'ruby', 'c++', 'html']})

TypeError: 'dict' object is not callable

In [ ]:
df

### 

## Load the mpg dataset as a spark dataframe.

- a. Create 1 column of output that contains a message like the one below:

The 1999 audi a4 has a 4 cylinder engine.

Do this for each vehicle

- b. Transform the trans column so that it only contains either manual or auto.

### 

In [4]:
# pull in the mpg dataset from data
mpg = data('mpg')
# transform that dataset into a spark dataframe
mpg = spark.createDataFrame(mpg)
# show me the top five of the new spark dataframe
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [39]:
mpg.select(
    F.concat(
        F.lit('The '), 
        (mpg.year).cast('string'), 
        F.lit(' '), 
        (mpg.model).cast('string'),
        F.lit(' '), 
        F.lit('has a'),
        F.lit(' '),
        (mpg.cyl).cast('string'),
        F.lit(' '),
        F.lit('cylinder engine')).alias('message')
).show(truncate=False)

+---------------------------------------------------+
|message                                            |
+---------------------------------------------------+
|The 1999 a4 has a 4 cylinder engine                |
|The 1999 a4 has a 4 cylinder engine                |
|The 2008 a4 has a 4 cylinder engine                |
|The 2008 a4 has a 4 cylinder engine                |
|The 1999 a4 has a 6 cylinder engine                |
|The 1999 a4 has a 6 cylinder engine                |
|The 2008 a4 has a 6 cylinder engine                |
|The 1999 a4 quattro has a 4 cylinder engine        |
|The 1999 a4 quattro has a 4 cylinder engine        |
|The 2008 a4 quattro has a 4 cylinder engine        |
|The 2008 a4 quattro has a 4 cylinder engine        |
|The 1999 a4 quattro has a 6 cylinder engine        |
|The 1999 a4 quattro has a 6 cylinder engine        |
|The 2008 a4 quattro has a 6 cylinder engine        |
|The 2008 a4 quattro has a 6 cylinder engine        |
|The 1999 a6 quattro has a 6

In [41]:
# it looks like we'll use regex here
mpg.select('trans').show()

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
|manual(m5)|
|  auto(av)|
|manual(m5)|
|  auto(l5)|
|manual(m6)|
|  auto(s6)|
|  auto(l5)|
|manual(m5)|
|  auto(s6)|
|manual(m6)|
|  auto(l5)|
|  auto(s6)|
|  auto(s6)|
|  auto(l4)|
|  auto(l4)|
+----------+
only showing top 20 rows



In [45]:
mpg.select(
    "trans",
    F.regexp_extract('trans', r'^(\w+)', 1).alias("city_state_zip")).show(truncate=False)

+----------+--------------+
|trans     |city_state_zip|
+----------+--------------+
|auto(l5)  |auto          |
|manual(m5)|manual        |
|manual(m6)|manual        |
|auto(av)  |auto          |
|auto(l5)  |auto          |
|manual(m5)|manual        |
|auto(av)  |auto          |
|manual(m5)|manual        |
|auto(l5)  |auto          |
|manual(m6)|manual        |
|auto(s6)  |auto          |
|auto(l5)  |auto          |
|manual(m5)|manual        |
|auto(s6)  |auto          |
|manual(m6)|manual        |
|auto(l5)  |auto          |
|auto(s6)  |auto          |
|auto(s6)  |auto          |
|auto(l4)  |auto          |
|auto(l4)  |auto          |
+----------+--------------+
only showing top 20 rows



## Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

### 

In [47]:
tips = data('tips')
tips = spark.createDataFrame(tips)
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [94]:
round((tips.select(tips.smoker).where(tips.smoker == 'Yes').count()) / (tips.select(tips.smoker).count())*100,2)

38.11

In [120]:
tips.select((F.round(F.expr('tip / total_bill')*100,2).alias('tip_percentage')), 'total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size').show()

+--------------+----------+----+------+------+---+------+----+
|tip_percentage|total_bill| tip|   sex|smoker|day|  time|size|
+--------------+----------+----+------+------+---+------+----+
|          5.94|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|         16.05|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|         16.66|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|         13.98|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|         14.68|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|         18.62|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|         22.81|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|         11.61|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|         13.03|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|         21.85|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|         16.65|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|         14.18|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|         10.18|     15.42|1.57|  Male|    No|Sun|Dinne

In [131]:
tips.groupBy('sex', 'smoker').agg(F.avg(F.round((tips.tip) / (tips.total_bill)*100,2)).alias('average_tip_percentage')).show(5)

+------+------+----------------------+
|   sex|smoker|average_tip_percentage|
+------+------+----------------------+
|  Male|    No|     16.06659793814433|
|Female|    No|     15.69111111111111|
|  Male|   Yes|    15.276666666666667|
|Female|   Yes|    18.214545454545455|
+------+------+----------------------+



## Use the Seattle weather dataset referenced in the lesson to answer the questions below.

In [132]:
from vega_datasets import data

In [165]:
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



### - Convert the temperatures to fahrenheit.

In [267]:
weather = weather.withColumn('temp_max', F.round(((weather.temp_max *9)/5)+32))
weather = weather.withColumn('temp_min', F.round(((weather.temp_min *9)/5)+32))

In [212]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|   144.0|   142.0| 4.7|drizzle|
|2012-01-02|         10.9|   142.0|   140.0| 4.5|   rain|
|2012-01-03|          0.8|   142.0|   142.0| 2.3|   rain|
|2012-01-04|         20.3|   144.0|   142.0| 4.7|   rain|
|2012-01-05|          1.3|   142.0|   140.0| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



### - Which month has the most rain, on average?

In [269]:
weather.withColumn(
    'month', F.month(F.col('date')
                    )
).groupBy(
    'month'
).agg(
    F.avg(weather.precipitation).alias('avg_rainfall')
).sort('avg_rainfall').show()

+-----+-------------------+
|month|       avg_rainfall|
+-----+-------------------+
|    7|0.38870967741935486|
|    6| 1.1075000000000002|
|    8| 1.3201612903225806|
|    5| 1.6733870967741935|
|    9| 1.9624999999999997|
|    4|  3.128333333333333|
|    2|  3.734513274336283|
|    1| 3.7580645161290316|
|   10|  4.059677419354839|
|    3|  4.888709677419355|
|   12|  5.021774193548389|
|   11|  5.354166666666667|
+-----+-------------------+



#### November has the most rainfall

### - Which year was the windiest?

In [199]:
weather.withColumn(
    'year', F.year(F.col('date')
                    )
).groupBy(
    'year'
).agg(
    F.max(weather.wind).alias('windy_days')
).sort('windy_days').show()

+----+----------+
|year|windy_days|
+----+----------+
|2015|       8.0|
|2013|       8.8|
|2014|       8.8|
|2012|       9.5|
+----+----------+



#### 2012 appears to be the windiest

### - What is the most frequent type of weather in January?

In [210]:
weather.withColumn(
    'month', F.month(F.col('date')
                    )
).groupBy(
    'month'
).agg(
    F.max(weather.weather).alias('weather')
).sort('weather').show()

+-----+-------+
|month|weather|
+-----+-------+
|    1|    sun|
|    2|    sun|
|    3|    sun|
|    4|    sun|
|    5|    sun|
|    6|    sun|
|    7|    sun|
|    8|    sun|
|    9|    sun|
|   10|    sun|
|   11|    sun|
|   12|    sun|
+-----+-------+



#### Sun

### - What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [228]:
weather = weather.withColumn('month', F.month(F.col('date')))
weather = weather.withColumn('year', F.year(F.col('date')))

In [229]:
weather.show()

+----------+-------------+--------+--------+----+-------+-----+----+
|      date|precipitation|temp_max|temp_min|wind|weather|month|year|
+----------+-------------+--------+--------+----+-------+-----+----+
|2012-01-01|          0.0|   144.0|   142.0| 4.7|drizzle|    1|2012|
|2012-01-02|         10.9|   142.0|   140.0| 4.5|   rain|    1|2012|
|2012-01-03|          0.8|   142.0|   142.0| 2.3|   rain|    1|2012|
|2012-01-04|         20.3|   144.0|   142.0| 4.7|   rain|    1|2012|
|2012-01-05|          1.3|   142.0|   140.0| 6.1|   rain|    1|2012|
|2012-01-06|          2.5|   140.0|   140.0| 2.2|   rain|    1|2012|
|2012-01-07|          0.0|   142.0|   140.0| 2.3|   rain|    1|2012|
|2012-01-08|          0.0|   142.0|   140.0| 2.0|    sun|    1|2012|
|2012-01-09|          4.3|   142.0|   142.0| 3.4|   rain|    1|2012|
|2012-01-10|          1.0|   142.0|   140.0| 3.4|   rain|    1|2012|
|2012-01-11|          0.0|   142.0|   138.0| 5.1|    sun|    1|2012|
|2012-01-12|          0.0|   142.0

In [280]:
# need average of both high and low temperatures 
# where weather = 'sunny'
# and in July of only 2013 and 2014
weather.filter(weather.weather == 'sun').where(weather.year.isin(2013, 2014)).where(weather.month == '7').agg(F.round(F.avg(weather.temp_max),2)).show()

+-----------------------+
|round(avg(temp_max), 2)|
+-----------------------+
|                 297.69|
+-----------------------+



In [231]:
# need average of both high and low temperatures 
# where weather = 'sunny'
# and in July of only 2013 and 2014
weather.filter(weather.weather == 'sun').where(weather.year == '2013').show()

+----------+-------------+--------+--------+----+-------+-----+----+
|      date|precipitation|temp_max|temp_min|wind|weather|month|year|
+----------+-------------+--------+--------+----+-------+-----+----+
|2013-01-01|          0.0|   140.0|   138.0| 2.7|    sun|    1|2013|
|2013-01-02|          0.0|   142.0|   138.0| 3.2|    sun|    1|2013|
|2013-01-12|          0.0|   140.0|   138.0| 2.0|    sun|    1|2013|
|2013-01-13|          0.0|   140.0|   138.0| 1.5|    sun|    1|2013|
|2013-01-14|          0.0|   140.0|   138.0| 1.3|    sun|    1|2013|
|2013-01-15|          0.0|   142.0|   140.0| 2.3|    sun|    1|2013|
|2013-02-08|          0.0|   142.0|   140.0| 1.3|    sun|    2|2013|
|2013-02-19|          0.0|   142.0|   140.0| 3.4|    sun|    2|2013|
|2013-03-03|          0.0|   142.0|   140.0| 2.8|    sun|    3|2013|
|2013-03-04|          0.0|   144.0|   140.0| 3.9|    sun|    3|2013|
|2013-03-17|          0.0|   142.0|   140.0| 6.1|    sun|    3|2013|
|2013-03-22|          0.0|   142.0

### - What percentage of days were rainy in Q3 of 2015?

### - For each year, find what percentage of days it rained (had non-zero precipitation).